In [12]:
# This script uses the OpenAI API in order to perform sentiment classification on Latin sentences
import json
import re
from openai import OpenAI
from openai._exceptions import RateLimitError
from dotenv import load_dotenv
import os
import time

load_dotenv()  # Load variables from .env into environment
api_key = os.getenv("API_KEY")

client = OpenAI(api_key=os.getenv("API_KEY"))

#Initialize the data
def init_data(filename: str) -> dict:
    with open(filename, "r", encoding="utf-8") as file:
        data_dict = json.load(file)
    return data_dict

def api_call(sentence: str) -> tuple[str,float]:
    try:
        completion = client.chat.completions.create(
        model = 'gpt-4.1',
        messages = [
        {'role': 'system', 'content': 'You are a chatbot.'},
        {'role': 'user', 'content': """You are a researcher on Latin literature. You will be given a Latin sentence.
 Do the following:
Analyze the sentence closely for its emotional content. Consider tone, imagery, word choice, and context.


Classify the sentence's emotionality using one or more of the following categories:


"love", "joy", "anger", "sadness", "fear", "shame", "disgust", "admiration", "agitation", "neutral"
Only include emotions that are clearly conveyed by the sentence. If the sentence is emotionally flat or objective, use "neutral".
Your output should be only a Python-style tuple of one or more lowercase strings (e.g., ("anger", "agitation")). Do not include explanations or extra text.
Examples:
Latin Sentence: “Utque animus tumida fervebat ab ira, / arma assueta rapit flexumque a cornibus arcum / tendit.”
 Answer: ("anger", "agitation")
Latin Sentence: “Sed tamen afueram, sed et haec erat unde redibam, / criminis exemplum, sed cuncta timemus amantes.”
 Answer: ("fear", "love")
Latin Sentence: “Ingemuit: vox illa fuit, lacrimaeque per ora / non sua fluxerunt; mens tantum pristina mansit.”
 Answer: ("sadness", "admiration")
Latin Sentence: “Qui postquam voce manuque / murmura compressit, tenuere silentia cuncti.”
 Answer: ("neutral")
"""},
        {'role': 'user', 'content': sentence}
        ])
        response_content = completion.choices[0].message.content
        return response_content
    except RateLimitError:
            print(f"Rate limit hit. Waiting 0.3s before retrying...")
            time.sleep(0.3)
    except Exception as e:
        print(f"Error processing sentence: {sentence}, Error: {e}")
        return ("Error in processing")
    
def sentiment_analysis(sentence_dict):
# Access the individual sentence in each dict
    for value in sentence_dict.values():
        for dictionary in value["occurrences"]:
            sentence = dictionary["sentence"]
            number = dictionary["number"]

            # Retrieve output of API call
            results = api_call(sentence)
            # Add sentiments to dict
            dictionary["sentiments"] = results

            # Add book number for easy calculations later
            hit = re.search("\. \d{1,2}", number)
            book_number = hit.group()
            dictionary["book_number"] = book_number[2:]
    return sentence_dict

def main():
    works = ["aeneid", "pharsalia", "thebaid", "argonautica", "punica"]
    for work in works:
        data = init_data(f"../analysis/{work}_dict.json")
        data = sentiment_analysis(data)
        with open(f"../analysis/{work}_sentiments.json", "w", encoding="utf-8") as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

main()

<>:72: SyntaxWarning: invalid escape sequence '\.'
<>:72: SyntaxWarning: invalid escape sequence '\.'
C:\Users\miwiy\AppData\Local\Temp\ipykernel_8208\3035865612.py:72: SyntaxWarning: invalid escape sequence '\.'
  hit = re.search("\. \d{1,2}", number)


Rate limit hit. Waiting 0.3s before retrying...
Rate limit hit. Waiting 0.3s before retrying...


In [ ]:
# Modified application on evaluation set
import csv

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()  # Load variables from .env into environment
api_key = os.getenv("API_KEY")

client = OpenAI(api_key=os.getenv("API_KEY"))

def api_call(sentence: str) -> str:
    try:
        completion = client.chat.completions.create(
        model = 'gpt-4.1',
        messages = [
        {'role': 'system', 'content': 'You are a chatbot.'},
        {'role': 'user', 'content': """You are a researcher on Latin literature. You will be given a Latin sentence.
 Do the following:
Analyze the sentence closely for its emotional content. Consider tone, imagery, word choice, and context.


Classify the sentence's emotionality using one or more of the following categories:


"love", "joy", "anger", "sadness", "fear", "shame", "disgust", "admiration", "agitation", "neutral"
Only include emotions that are clearly conveyed by the sentence. If the sentence is emotionally flat or objective, use "neutral".
Your output should be only a Python-style tuple of one or more lowercase strings (e.g., ("anger", "agitation")). Do not include explanations or extra text.
Examples:
Latin Sentence: “Utque animus tumida fervebat ab ira, / arma assueta rapit flexumque a cornibus arcum / tendit.”
 Answer: ("anger", "agitation")
Latin Sentence: “Sed tamen afueram, sed et haec erat unde redibam, / criminis exemplum, sed cuncta timemus amantes.”
 Answer: ("fear", "love")
Latin Sentence: “Ingemuit: vox illa fuit, lacrimaeque per ora / non sua fluxerunt; mens tantum pristina mansit.”
 Answer: ("sadness", "admiration")
Latin Sentence: “Qui postquam voce manuque / murmura compressit, tenuere silentia cuncti.”
 Answer: ("neutral")
"""},
        {'role': 'user', 'content': sentence}
        ])
        response_content = completion.choices[0].message.content
        return response_content
    except Exception as e:
        print(f"Error processing sentence: {sentence}, Error: {e}")
        return ("Error in processing")
    
def load_data():
    data = []
    with open("../analysis/sampled_sentences.txt", "r", encoding="utf-8") as file:
            for line in file:
                data.append(line.strip().split("\t"))
    return data

def main():
     eval_set = load_data()
     for line in eval_set:
          result = api_call(line[1])
          line.append(result)
    
     with open("../analysis/eval_predictions.txt", 'w', newline='', encoding='utf-8') as file:
        output = csv.writer(file, delimiter='\t', quotechar='|')
        for i in eval_set:
            if len(i) > 1:
                output.writerow(i)

main()
